In [1]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2025-08-12 04:07:45.858222: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 04:07:45.874286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 04:07:45.899491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 04:07:45.899529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 04:07:45.915241: I tensorflow/core/platform/cpu_feature_gua

2.16.2


In [7]:
imageFileNames = tf.data.Dataset.list_files('./Images/*', shuffle=False)

labelDict = {}
with open("./dev_data_2025.csv", "r") as f:
    for imgLabel in csv.DictReader(f):
        labelDict[imgLabel['imageID']] = [
            imgLabel['label'], 
            imgLabel['cell_shape'], 
            imgLabel['nucleus_shape'], 
            imgLabel['cytoplasm_vacuole']
        ]

In [10]:
print(labelDict['Img_00003'])
for f in imageFileNames.take(5):
    print(f.().to_str().split('/'))


['Monocyte', 'irregular', 'irregular', 'yes']


AttributeError: 'bytes' object has no attribute 'to_str'

In [ ]:
def process_path(file_path):
    
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img_decode = tf.io.decode_jpeg(img, channels=3)
  return img_decode, label
